In [ ]:

import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import os

2121 596


In [ ]:
path = '/content/drive/MyDrive/cs550/ml-latest-small'
ratings = pd.read_csv(os.path.join(path, "ratings.csv"), sep=",", nrows=None)
tags = pd.read_csv(os.path.join(path, "tags.csv"), sep=",", nrows=None)
movies = pd.read_csv(os.path.join(path, "movies.csv"), sep=",", nrows=None)

# final_dataset = ratings.pivot(index='movieId',columns='userId',values='rating')
# final_dataset.fillna(0,inplace=True)
# no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
# no_movies_voted = ratings.groupby('userId')['rating'].agg('count')

# final_dataset = final_dataset.loc[no_user_voted[no_user_voted > 10].index,:]
# final_dataset = final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
# final_dataset.head()

movie_count = ratings['movieId'].value_counts().reset_index().rename(columns={'index':'movieId', 'movieId': 'count'})
user_count = ratings['userId'].value_counts().reset_index().rename(columns={'index':'userId', 'userId': 'count'})
movie_count.head()
user_count.head()
userSet = set(user_count[user_count['count'] > 20]['userId'])
movieSet = set(movie_count[movie_count['count'] > 10]['movieId'])
print(len(movieSet), len(userSet))

In [ ]:
import re

pattern = '\(\d\d\d\d\)'

movie_title = movies['title']

# title = movie_title.map(lambda x: re.sub(pattern, " ", x))
# year = movie_title.map(lambda x: re.findall("\d\d\d\d", x)[-1] if re.search("\d\d\d\d", x) is not None else "0000")
# print(year)
movie_year = {}
movie_dict = {}
movieIdset = set(movies["movieId"])
for movieId in movieIdset:
  sub_tags = tags[tags['movieId']==movieId]
  sub_movies = movies[movies['movieId']==movieId]
  if movieId not in movie_dict:
    movie_dict[movieId] = {}
  title = sub_movies['title'].item()
  movie_dict[movieId]['title'] = re.sub(pattern, " ", title)
  movie_dict[movieId]['genres'] = list(sub_movies['genres'])[0].replace("|", " ")
  if len(sub_tags) > 0:
    if "tag" not in movie_dict[movieId]:
      movie_dict[movieId]['tag'] = ""
    movie_dict[movieId]['tag'] += " ".join(list(sub_tags['tag']))
  movie_year[movieId] = int(re.findall("\d\d\d\d", title)[-1]) if re.search("\d\d\d\d", title) is not None else 1800
# print(movie_dict)
# print(movie_year)
movie_text = {}
for movieId, info in movie_dict.items():
  movie_text[movieId] = info['title'].strip() + " " + info['genres'].strip() + ((" tags "+" ".join(set(info['tag'].strip().split(" ")))) if 'tag' in info else "")

378


In [ ]:
ratings_dataset = ratings.pivot(index='movieId',columns='userId',values='rating')
ratings_dataset.fillna(0,inplace=True)
# ratings_dataset = ratings_dataset.reset_index()
no_user_voted = ratings.groupby('movieId')['rating'].agg('count')
no_movies_voted = ratings.groupby('userId')['rating'].agg('count')
final_dataset = ratings_dataset.loc[no_user_voted[no_user_voted > 10].index,:]
final_dataset = final_dataset.loc[:,no_movies_voted[no_movies_voted > 50].index]
final_dataset = final_dataset.reset_index()
# final_dataset.shape
movie_user_dict = final_dataset.set_index('movieId').T.to_dict('list')
print(len(movie_user_dict[1]))

In [ ]:
!pip install transformers

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from transformers import BertModel, AdamW, BertConfig, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

input_ids = []
attention_masks = []

movie_token = {}
for movieId, text in movie_dict.items():
  movie_token[movieId] = {}
  encoded_dict = tokenizer.encode_plus(
    text=movie_text[movieId],                      # Sentence to encode.
    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
    max_length = 210,           # Pad & truncate all sentences.
    pad_to_max_length = True,
    return_attention_mask = True,   # Construct attn. masks
    return_tensors = 'pt',     # Return pytorch tensors.
  )
  movie_token[movieId]['input_ids'] = encoded_dict['input_ids']
  movie_token[movieId]['attention_mask'] = encoded_dict['attention_mask']

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:

movie_vec = {}
model = BertModel.from_pretrained("bert-base-uncased")

for movieId in movieSet:
  tokens = movie_token[movieId]
  CLS = model(tokens['input_ids'], token_type_ids=None, attention_mask=tokens['attention_mask'])[1]
  movie_vec[movieId] = CLS.detach().numpy().reshape(-1)
  # print(movie_vec[movieId].shape)

In [ ]:
import pickle as pkl

# with open(os.path.join(path, "movie_vec.pkl"), 'wb') as f:
#   pkl.dump(movie_vec, f)

In [ ]:
with open(os.path.join(path, "movie_vec.pkl"), 'rb') as f:
  movie_vec = pkl.load(f)
# for movieId in movie_vec:
#   movie_vec[movieId] = np.append(CLS.detach().numpy().reshape(-1), np.array(movie_user_dict[movieId]))
# print(len(movie_vec))

(2121, 769)

In [ ]:
data = pd.DataFrame.from_dict(movie_vec, orient='index')
csr_data = csr_matrix(data.values)
# data = data.reset_index(inplace=True)
data = data.reset_index().rename(columns={'index':'movieId'})
data.shape
# data.index

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=15)

In [ ]:

knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=15, n_jobs=-1)
knn.fit(csr_data)

In [ ]:
def get_movie_recommendation(movie_name, num_rec):
    n_movies_to_reccomend = num_rec
    movie_list = movies[movies['title'].str.contains(movie_name)]  
    if len(movie_list):        
        movieIdx= movie_list.iloc[0]['movieId']
        # print(movieIdx)
        vec = movie_vec[movieIdx]
        movie_idx = data[data['movieId'] == movieIdx].index[0]
        # print(movie_idx)
        distances , indices = knn.kneighbors(csr_data[movie_idx],n_neighbors=n_movies_to_reccomend+1)    
        rec_movie_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
        recommend_frame = []
        for val in rec_movie_indices:
            movie_idx = data.iloc[val[0]]['movieId']
            idx = movies[movies['movieId'] == movie_idx].index
            recommend_frame.append({'Title':movies.iloc[idx]['title'].values[0],'Distance':val[1]})
        df = pd.DataFrame(recommend_frame,index=range(1,n_movies_to_reccomend+1))
        return df
    else:
        return "No movies found. Please check your input"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


'No movies found. Please check your input'

In [ ]:
get_movie_recommendation("AvengersUltimate  (2006)", 20)